In [1]:
__NAME = 'Harmonic 1'

# Imports

In [2]:
import tensorflow as tf
print(tf.version)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

<module 'tensorflow._api.v1.version' from 'I:\\Users\\Ragav\\miniconda3\\envs\\juptenflowgpu115\\lib\\site-packages\\tensorflow_core\\_api\\v1\\version\\__init__.py'>
True
True


In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# from plotting import newfig, savefig
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
import numpy as np
import scipy.io
from scipy.interpolate import griddata
import time
from pyDOE import lhs

In [5]:
import pickle as pkl

In [6]:
%matplotlib widget

# Equation

In [7]:
k = 1

$$ y'' + k^2 y = 0 $$

# Model 

In [8]:
# Layers
u_layers = [1, 50, 50, 50, 50, 1]
pde_layers = [3, 100, 100, 1]

layers = [1, 50, 50, 50, 50, 1]

# layers = [1, 100, 1000, 1000, 100, 1]

# layers = [1, 3, 3, 3, 1]

In [9]:
x_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_tf, u_tf

(<tf.Tensor 'Placeholder:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>)

In [10]:
def initialize_NN(layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l + 1]])
        b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                        dtype=tf.float32)
        weights.append(W)
        biases.append(b)
    return weights, biases

def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim],
                                           stddev=xavier_stddev,
                                           dtype=tf.float32),
                       dtype=tf.float32)

In [11]:
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    H = X
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.sin(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [12]:
u_weights, u_biases = initialize_NN(layers)

In [13]:
# Load Weights and biases

# npz1 = np.load('Harmonic 1\\weights.npz')
# npz2 = np.load('Harmonic 1\\biases.npz')

with open(__NAME + '\\weights.pkl', 'rb') as db_file:
    W_pkl = pkl.load(db_file)

with open(__NAME + '\\biases.pkl', 'rb') as db_file:
    B_pkl = pkl.load(db_file)

W = []
B = []
for w, b in zip(W_pkl, B_pkl):
    W.append(tf.Variable(w))
    B.append(tf.Variable(b))

u_weights = W
u_biases = B

In [14]:
lb_tf = tf.placeholder(tf.float32, shape=[1])
ub_tf = tf.placeholder(tf.float32, shape=[1])

In [15]:
# tf placeholders for Solution
x0_tf = tf.placeholder(tf.float32, shape=[None, 1])
u0_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_x0_tf = tf.placeholder(tf.float32, shape=[None, 1])

x_f_tf = tf.placeholder(tf.float32, shape=[None, 1])

In [16]:
def sol_net_u(x):
    X = tf.concat([x], 1)
    H = 2.0 * (X - lb_tf) / (ub_tf - lb_tf) - 1.0
    u = neural_net(H, u_weights, u_biases)
    u_x = tf.gradients(u, x)[0]
    return u, u_x

def sol_net_f(x):
    u, u_x = sol_net_u(x)

    u_xx = tf.gradients(u_x, x)[0]

    f = u_xx + k**2 * u

    return f

In [17]:
# tf graphs for Solution
u0_pred, u_x0_pred = sol_net_u(x0_tf)

sol_f_pred = sol_net_f(x_f_tf)

In [18]:
# loss for Solution
sol_loss = tf.reduce_sum(tf.square(u0_tf - u0_pred)) + \
           tf.reduce_sum(tf.square(u_x0_pred - u_x0_tf)) + \
           tf.reduce_sum(tf.square(sol_f_pred))

In [19]:
# Optimizer for Solution
adam_optimizer = tf.train.AdamOptimizer()
sol_train_op_Adam = adam_optimizer.minimize(
            sol_loss,
            var_list= u_weights + u_biases)

sol_optimizer = tf.contrib.opt.ScipyOptimizerInterface(
    sol_loss,
    var_list=u_weights + u_biases,
    method='L-BFGS-B',
    options={
        'maxiter': 50000,
        'maxfun': 50000,
        'maxcor': 50,
        'maxls': 50,
        'ftol': 1.0 * np.finfo(float).eps
    })

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [20]:
# tf session
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



# Training
## Prepare data

In [21]:
lb = np.array([- np.pi * 2])
ub = np.array([np.pi * 2])

In [22]:
x0 = np.array([0])
u0 = np.array([0])
u_x0 = np.array([1])

In [23]:
N_f = 1000
(N_f)

1000

In [24]:
X_f_train = lb + (ub - lb) * lhs(1, N_f) 

In [25]:
def callback(loss):
    print('Loss: %e' % (loss))

In [26]:
tf_dict = {
    lb_tf: lb,
    ub_tf: ub,
    x0_tf: x0[None, :],
    u0_tf: u0[None, :],
    u_x0_tf: u_x0[None, :],
    x_f_tf: X_f_train
}

In [27]:
start_time = time.time()
for it in range(1000):

    sess.run(sol_train_op_Adam, tf_dict)

    # Print
    if it % 10 == 0:
        elapsed = time.time() - start_time
        loss_value = sess.run(sol_loss, tf_dict)
        print('It: %d, Loss: %.3e, Time: %.2f' %
                (it, loss_value, elapsed))
        start_time = time.time()

It: 0, Loss: 7.865e+00, Time: 4.21
It: 10, Loss: 4.807e-01, Time: 0.12
It: 20, Loss: 5.893e-01, Time: 0.11
It: 30, Loss: 1.646e-01, Time: 0.10
It: 40, Loss: 1.593e-02, Time: 0.10
It: 50, Loss: 3.067e-03, Time: 0.10
It: 60, Loss: 1.587e-03, Time: 0.09
It: 70, Loss: 1.772e-03, Time: 0.09
It: 80, Loss: 1.906e-03, Time: 0.10
It: 90, Loss: 1.579e-03, Time: 0.10
It: 100, Loss: 9.812e-04, Time: 0.09
It: 110, Loss: 8.047e-04, Time: 0.10
It: 120, Loss: 7.112e-04, Time: 0.10
It: 130, Loss: 5.973e-04, Time: 0.10
It: 140, Loss: 5.180e-04, Time: 0.09
It: 150, Loss: 4.433e-04, Time: 0.10
It: 160, Loss: 3.790e-04, Time: 0.09
It: 170, Loss: 3.230e-04, Time: 0.10
It: 180, Loss: 2.741e-04, Time: 0.10
It: 190, Loss: 2.319e-04, Time: 0.11
It: 200, Loss: 1.956e-04, Time: 0.11
It: 210, Loss: 1.647e-04, Time: 0.12
It: 220, Loss: 1.386e-04, Time: 0.10
It: 230, Loss: 1.166e-04, Time: 0.10
It: 240, Loss: 9.822e-05, Time: 0.10
It: 250, Loss: 8.301e-05, Time: 0.10
It: 260, Loss: 7.049e-05, Time: 0.10
It: 270, Los

In [28]:
sol_optimizer.minimize(sess,
                       feed_dict=tf_dict,
                       fetches=[sol_loss],
                       loss_callback=callback)

Loss: 6.557041e-06
Loss: 2.787528e+03
Loss: 6.557086e-06
Loss: 6.557059e-06
Loss: 6.556744e-06
Loss: 6.556782e-06
Loss: 6.556744e-06
Loss: 6.556744e-06
Loss: 6.556654e-06
Loss: 1.219565e-05
Loss: 6.556790e-06
Loss: 6.556639e-06
Loss: 6.556246e-06
Loss: 6.555391e-06
Loss: 6.553398e-06
Loss: 6.545834e-06
Loss: 6.528980e-06
Loss: 6.508954e-06
Loss: 6.488440e-06
Loss: 6.438780e-06
Loss: 6.396902e-06
Loss: 6.315632e-06
Loss: 6.202415e-06
Loss: 6.195025e-06
Loss: 6.158148e-06
Loss: 6.108374e-06
Loss: 6.080323e-06
Loss: 6.054768e-06
Loss: 6.009881e-06
Loss: 5.958202e-06
Loss: 5.932652e-06
Loss: 5.906897e-06
Loss: 5.828116e-06
Loss: 5.687250e-06
Loss: 5.318551e-06
Loss: 4.807518e-06
Loss: 4.369186e-06
Loss: 4.228767e-06
Loss: 4.199263e-06
Loss: 4.192795e-06
Loss: 4.179072e-06
Loss: 4.155274e-06
Loss: 4.105073e-06
Loss: 4.049029e-06
Loss: 4.020072e-06
Loss: 4.005873e-06
Loss: 4.001682e-06
Loss: 4.001013e-06
Loss: 3.997727e-06
Loss: 3.994647e-06
Loss: 3.993301e-06
Loss: 3.993266e-06
Loss: 3.9931

In [29]:
sess.run(sol_loss, feed_dict=tf_dict)

3.9901292e-06

In [30]:
# np.savez('Harmonic 1\\weights.npz', sess.run(u_weights), dtype=object, allow_pickle=True)
# np.savez('Harmonic 1\\biases.npz', sess.run(u_biases), allow_pickle=True)
with open(__NAME + '\\weights.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(u_weights), file=db_file)

with open(__NAME + '\\biases.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(u_biases), file=db_file)

In [31]:
x_sol = np.linspace(lb[0], ub[0], num=1000)

In [32]:
u_pred = sess.run(u0_pred, {
        lb_tf: lb,
        ub_tf: ub,
        x0_tf: x_sol[:, None]
    })

# loss function
f_pred = sess.run(sol_f_pred, {
        lb_tf: lb,
        ub_tf: ub,
        x_f_tf: x_sol[:, None]
    })

In [33]:
# fig, ax = plt.subplots()
# ax.set_xlabel('$x$')
# ax.set_ylabel('$y$')
# ax.plot(x_sol, u_pred)
# fig.set_figheight(3.2)
# fig.set_figwidth(6)
# plt.tight_layout()

In [1]:
fig = plt.figure(figsize=(4*1.3333,4), dpi=200)
ax = fig.gca()

NameError: name 'plt' is not defined

In [49]:
ax.set_xlim(lb[0], ub[0])
# ax.set_xticks(np.arange(lb[0],ub[0],(ub[0] - lb[0])/N))
# ax.set_yticks(np.arange(lb[1],ub[1],(ub[1] - lb[1])/N))

ax.yaxis.grid(color='gainsboro', linestyle='dotted', linewidth=1.5)
ax.xaxis.grid(color='gainsboro', linestyle='dotted', linewidth=0.8)
ax.axhline(0,linestyle='dotted', color='grey')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
# plt.subplots_adjust(bottom=0.17)
# plt.subplots_adjust(left=0.17)

plt.title('Simple Harmonic Oscillator')
plt.xlabel('$x$')
plt.ylabel('$y$')

ax.plot(x_sol, u_pred, color ='red', label='PINN sol')
ax.plot(x_sol, np.sin(k * x_sol) / k, color ='blue', label='ideal sol', ls='-.')

plt.tight_layout()

plt.legend()

# fig.show()
fig.savefig('Figures\\SHO.png')

In [36]:
# fig, ax = plt.subplots()

In [37]:
# ax.plot(x_sol, u_pred)
# # ax.plot(x_sol, f_pred)